In [1]:
%pip install fastai
%pip install torch torchvision torchaudio


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 2.4/2.4 MB 10.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from fastai.vision.all import *
import pathlib

# Stel het pad in naar je afbeeldingen
data_dir = pathlib.Path("/Screenshots")

# DataBlock voor het verwerken van afbeeldingen
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),  # We classificeren afbeeldingen
    get_items=get_image_files,           # Alle afbeeldingsbestanden ophalen
    splitter=RandomSplitter(valid_pct=0.2, seed=123),  # 20% voor validatie
    get_y=parent_label,                  # Labels worden gebaseerd op mappenamen
    item_tfms=Resize(180),               # Afbeeldingen worden geschaald naar 180x180
    batch_tfms=aug_transforms(size=180)  # Basisaugmentatie en schaling
)

# Maak een DataLoaders object
dls = dblock.dataloaders(data_dir, bs=32)

# Het model trainen met een vooraf getrainde ResNet-architectuur
learn = cnn_learner(dls, resnet18, metrics=accuracy)

# Samenvatting van het model
learn.model

# Optionele callback voor vroegtijdig stoppen
early_stopping_cb = EarlyStoppingCallback(monitor='valid_loss', patience=8)

# Train het model
learn.fit_one_cycle(20, cbs=[early_stopping_cb])

# Bewaar het getrainde model
learn.export('ad-model.h5')
